In [ ]:
import numpy as np
import pandas as pd

from bokeh.layouts import column as bkhcolumn, row as bkhrow
from bokeh.io import show

import bokeh.models as bkm
import bokeh.plotting as bkp
# import matplotlib.pyplot as plt\
# import mpltern

import warnings
warnings.filterwarnings('ignore')

In [ ]:
party_to_colour = {
    'ALP': '#E50000',
    "ALPN": "#AF1818",
    'LP': '#0000FF', 
    'NP': "#1B7200",
    'CLP': '#0000FF',
    'CP': '#1B7200',
    'NPA': '#1B7200',
    'NAT': '#1B7200',
    'NCP': '#1B7200',
    'LNQ': "#0000FF",
    'LNP': '#0000FF',
    'GRN': "#00D600",
    'ON': '#FF8000',
    'KAP': "#804000",
    'UAP': "#CFD300",
    'PUP': "#CFD300",
    'IND': "#777777",
    'CA': "#777777",
    'NXT': "#777777",
    'XEN': "#777777",
}

def ternary_to_cartesian(l, r, t):
    """
    Convert ternary coordinates to Cartesian coordinates for plotting.
    """
    y = np.sqrt(3) / 2 * t / (l + r + t)
    x = 0.5 * (2*r + t) / (l + r + t)
    return x, y

def df_to_tooltip(df, year=None):
    title = df['Division'].iloc[0] + ", " + df['State'].iloc[0]
    if year is not None:
        title += ", " + year
    html = f'<div class="vote-tooltip"><h3 style="margin-top:0">{title}</h3><table>'
    html += '<tr><th>Party</th><th>Vote Share</th><th>Candidate</th></tr>'
    for _, row in df.sort_values('Votes', ascending=False, inplace=False).iterrows():
        html += f'<tr><td>{row["Party"]}</td><td>{row["Percent"]:.2f}</td><td>{row["Candidate"]}</td></tr>'
    return html + '</table></div>'

def df_to_coordinates(df):
    """
    Convert a table of party votes to a coordinate for a triangular plot.
    """
    n_votes = {
        'ALP': 0,
        'ALPN': 0,
        'LP': 0, 
        'NP': 0,
        'CLP': 0,
        'LNQ': 0,
        'LNP': 0,
        'CP': 0,
        'NPA': 0,
        'NAT': 0,
        'NCP': 0,
    }
    for party in n_votes.keys():
        if party in df['Party'].values:
            n_votes[party] = df.query(f'`Party`=="{party}"')['Percent'].iloc[0] / 100
    l = n_votes['ALP'] + n_votes['ALPN']
    r = (
        n_votes['LP'] 
        + n_votes['NP'] 
        + n_votes['CLP'] 
        + n_votes['LNQ'] 
        + n_votes['LNP'] 
        + n_votes['CP'] 
        + n_votes['NPA'] 
        + n_votes['NAT'] 
        + n_votes['NCP']
    )
    t = 1 - l - r
    return ternary_to_cartesian(l, r, t)
    # return t, l, r

In [ ]:
house_members_by_election = {
    "1901": 75, "1903": 75, "1906": 75, "1910": 75, "1913": 75,
    "1914": 75,    "1917": 75,    "1919": 75,    "1922": 76,    "1925": 76,
    "1928": 76,    "1929": 76,    "1931": 76,    "1934": 75,    "1937": 75,
    "1940": 75,    "1943": 75,    "1946": 75,    "1949": 123,    "1951": 123,
    "1954": 123,    "1955": 124,    "1958": 124,    "1961": 124,    "1963": 124,
    "1966": 124,    "1969": 125,    "1972": 125,    "1974": 127,    "1975": 127,
    "1977": 124,    "1980": 125,    "1983": 125,    "1984": 148,    "1987": 148,
    "1990": 148,    "1993": 147,    "1996": 148,    "1998": 148,    "2001": 150,    
    "2004": 150,    "2007": 150,    "2010": 150,    "2013": 150,    "2016": 150,    
    "2019": 151,    "2022": 151,    "2025": 150,
}

In [ ]:
election_years = [
    '1946', '1949', '1951', '1954', '1955', '1958', '1961', 
    '1963', '1966', '1969', '1972', '1974', '1975', '1977',
    '1980', '1983', '1987', '1990', '1993', '1996', '1998', '2001',
    '2004', '2007', '2010', '2013', '2016', '2019', '2022', '2025'
]

year_dict = {}

for year in election_years:

    first_prefs = pd.read_csv(f'Data\\First Prefs\\first_prefs_{year}.csv')
    winners = pd.read_csv(f'Data\\Winners\\winner_{year}.csv')

    if (
        len(winners) != house_members_by_election[year] 
        or len(first_prefs['Division'].unique()) != house_members_by_election[year]
    ):
        print(year)
        print(f"First prefs {len(first_prefs['Division'].unique())}")
        print(f"Winners: {len(winners)}")
        print(f"Expected: {house_members_by_election[year]}")
        if any(winners['Division'].duplicated()):        
            print(winners[winners['Division'].duplicated()])

    colour = winners.set_index('Division')['Party'].map(lambda s: s.strip()).map(party_to_colour).rename("color", axis=0)
    coords = first_prefs.groupby("Division").apply(df_to_coordinates).rename("coords", axis=0)
    x = coords.apply(lambda tup: tup[0]).rename("x")
    y = coords.apply(lambda tup: tup[1]).rename("y")
    # tooltip = first_prefs.groupby("Division").apply(df_to_tooltip, year=year).rename("tooltip", axis=0)
    state = winners.set_index('Division')['State'].rename("State")
    party = first_prefs.groupby("Division").apply(lambda df: df.sort_values('Percent', ascending=False)['Party'].values).rename("Party", axis=0)
    percent = first_prefs.groupby("Division").apply(lambda df: df.sort_values('Percent', ascending=False)['Percent'].values).rename("Percent", axis=0)
    candidate = first_prefs.groupby("Division").apply(lambda df: df.sort_values('Percent', ascending=False)['Candidate'].values).rename("Candidate", axis=0)

    year_dict[year] = pd.concat(
        [    
            x,
            y,
            colour,
            state,
            party,
            percent,
            candidate
        ],
        axis=1,
    ).reset_index()

In [ ]:
# all_divisions = set()
# for year, df in year_dict.items():
#     all_divisions.update(df['Division'])

# # Build the dictionary
# division_dict = {}
# for division in all_divisions:
#     records = []
#     for year, df in year_dict.items():
#         if division in df['Division'].values:
#             row = df.query(f'`Division`=="{division}"').copy()
#             row['year'] = year
#             records.append(row)
#     # print(records)
#     if records:
#         division_df = pd.concat(records, ignore_index=True, axis=0)
#         division_dict[division] = division_df

In [ ]:
#Background
N = 500
Y, X = np.mgrid[0:1:N*1j, 0:1:N*1j]

# Ternary Coordinates
L = 1 - X - 1/np.sqrt(3) * Y
R = X - 1/np.sqrt(3) * Y
T = 2 / np.sqrt(3) * Y

img_rgba = np.zeros((N, N, 4), dtype=np.uint8)

#Left
ind = (L > R) & (L > T) & (L >= 0) & (R >= 0) & (T >= 0)
img_rgba[:, :, 0] = img_rgba[:, :, 0] + 255 * ind * L  # Red channel
# img_rgba[:, :, 1] = 0    # Green channel
# img_rgba[:, :, 2] = 0    # Blue channel
img_rgba[:, :, 3] = img_rgba[:, :, 3] + 150 * ind * L  # Alpha channel

#Right
ind = (R > L) & (R > T) & (R >= 0) & (L >= 0) & (T >= 0)
# img_rgba[:, :, 0] = 0    # Red channel
# img_rgba[:, :, 1] = 0  # Green channel
img_rgba[:, :, 2] = img_rgba[:, :, 2] + 255 * ind * R    # Blue channel
img_rgba[:, :, 3] = img_rgba[:, :, 3] + 150 * ind * R  # Alpha channel

#Top
ind = (T > L) & (T > R) & (T >= 0) & (L >= 0) & (R >= 0)
img_rgba[:, :, 0] = img_rgba[:, :, 0] + 255/2 * ind * T  # Red channel
img_rgba[:, :, 1] = img_rgba[:, :, 1] + 255/2 * ind * T  # Green channel
img_rgba[:, :, 2] = img_rgba[:, :, 2] + 255/2 * ind * T  # Blue channel
img_rgba[:, :, 3] = img_rgba[:, :, 3] + 150 * ind * T  # Alpha channel

# Convert RGBA image to uint32 as required by Bokeh
img_view = img_rgba.view(dtype=np.uint32).reshape((N, N))

In [ ]:
default_year = "2025"  # Set the default year for the plot

# Prepare figure
p = bkp.figure(width=800, height=800, toolbar_location=None)

# Draw ternary background (reuse img_view)
p.image_rgba(image=[img_view], x=0, y=0, dw=1, dh=1)
p.line([0, 1, 0.5, 0], [0, 0, np.sqrt(3)/2, 0], line_color='black', line_width=2)
p.line([1/4, 0.5, 3/4], [np.sqrt(3)/4, 0, np.sqrt(3)/4], line_color='black', line_width=1, alpha=0.2)

# Hide axes
p.grid.visible = False
p.xaxis.visible = False
p.yaxis.visible = False
p.x_range = bkm.Range1d(0, 1)
p.y_range = bkm.Range1d(0, 1)

# Data source for current year
# source = bkm.ColumnDataSource(data=year_dict[default_year])

source = bkm.ColumnDataSource(data={
    'division': year_dict[default_year]['Division'].tolist(),
    'x': year_dict[default_year]['x'].tolist(),
    'y': year_dict[default_year]['y'].tolist(),
    'color': year_dict[default_year]['color'].tolist(),
    'tooltip': [
        # Build tooltips on the fly for each row
        f'<div class="vote-tooltip"><h3 style="margin-top:0">{row["Division"]}, {row["State"]}, {default_year}</h3><table>'
        + '<tr><th>Party</th><th>Vote Share</th><th>Candidate</th></tr>'
        + ''.join(
            f'<tr><td>{p}</td><td>{v:.2f}</td><td>{c}</td></tr>'
            for p, v, c in zip(row["Party"], row["Percent"], row["Candidate"])
        )
        + '</table></div>'
        for _, row in year_dict[default_year].iterrows()
    ],
    'state': year_dict[default_year]['State'].tolist(),
    'party': year_dict[default_year]['Party'].tolist(),
    'percent': year_dict[default_year]['Percent'].tolist(),
    'candidate': year_dict[default_year]['Candidate'].tolist(),
    'year': [default_year] * len(year_dict[default_year]),
    'alpha': [1] * len(year_dict[default_year]),
    'line_alpha': [1] * len(year_dict[default_year]),
})

change_line_source = bkm.ColumnDataSource(data=dict(xs=[], ys=[]))
line_glyph = p.multi_line(xs='xs', ys='ys', line_color='black', line_dash='dashed', line_width=1, alpha=0.7, source=change_line_source)

# Scatter glyph
g = bkm.Circle(
    x='x', y='y',
    fill_color='color',
    line_color='black',
    line_width=0.5,
    radius=0.0035,
    fill_alpha='alpha',
    line_alpha='line_alpha'  # <-- per-point line alpha
)
r = p.add_glyph(source, g)
r.nonselection_glyph = g
r.name = "main_scatter"

# Hover tool
hover = bkm.HoverTool(renderers=[r], tooltips="@tooltip")
p.add_tools(hover)

# Tap tool for selecting divisions
tap = bkm.TapTool(renderers=[r])
p.add_tools(tap)

selected_division_source = bkm.ColumnDataSource(data=dict(x=[], y=[], year=[], color=[], tooltip=[]))
selected_line_source = bkm.ColumnDataSource(data=dict(xs=[], ys=[]))
selected_line_glyph = p.multi_line(xs='xs', ys='ys', line_color='black', line_dash='dashed', line_width=1, alpha=0.7, source=selected_line_source)
selected_glyph = p.circle(
    x='x', y='y',
    color='color',
    radius=0.0035,
    alpha=1,
    line_width=0.5,           # Make border more visible if you like
    line_color='black',       # <-- add this
    line_alpha=1,             # <-- and this (optional, for clarity)
    source=selected_division_source
)

selected_hover = bkm.HoverTool(renderers=[selected_glyph], tooltips="@tooltip")
p.add_tools(selected_hover)

# # Widgets
show_changing = bkm.CheckboxGroup(labels=["Show Changing Seats"], active=[])
select = bkm.Select(value=default_year, options=election_years)
button_prev = bkm.Button(label="⟨", width=40)
button_next = bkm.Button(label="⟩", width=40)

# JS data for all years
js_data_year = {y: {
    'division': year_dict[y]['Division'].tolist(),
    'x': year_dict[y]['x'].tolist(),
    'y': year_dict[y]['y'].tolist(),
    'color': year_dict[y]['color'].tolist(),
    'state': year_dict[y]['State'].tolist(),
    'party': year_dict[y]['Party'].tolist(),
    'percent': year_dict[y]['Percent'].tolist(),
    'candidate': year_dict[y]['Candidate'].tolist(),
    # 'tooltip': year_dict[y]['tooltip'].tolist()
} for y in year_dict}

# js_data_division = {division: {
#     'year': division_dict[division]['year'].tolist(),
#     'x': division_dict[division]['x'].tolist(),
#     'y': division_dict[division]['y'].tolist(),
#     'color': division_dict[division]['color'].tolist(),
#     'tooltip': division_dict[division]['tooltip'].tolist()
# } for division in division_dict.keys()}

# JS callback for tap
p.js_on_event('tap', bkm.CustomJS(
    args=dict(
        source=source,
        js_data_year=js_data_year,
        selected_division_source=selected_division_source,
        selected_line_source=selected_line_source,
        select=select,
        show_changing=show_changing,
        change_line_source=change_line_source,
        election_years=election_years,  # <-- add this
    ),
    code="""
    let indices = source.selected.indices;
    let d = source.data;
    const current_year = d.year[0];
    const year_data = js_data_year[current_year];

    function makeTooltip(division, state, year, parties, percents, candidates) {
        let html = `<div class="vote-tooltip"><h3 style="margin-top:0">${division}, ${state}, ${year}</h3><table>`;
        html += '<tr><th>Party</th><th>Vote Share</th><th>Candidate</th></tr>';
        for (let i = 0; i < parties.length; i++) {
            html += `<tr><td>${parties[i]}</td><td>${percents[i].toFixed(2)}</td><td>${candidates[i]}</td></tr>`;
        }
        html += '</table></div>';
        return html;
    }

    function resetAllPoints() {
        for (let i = 0; i < d.division.length; i++) {
            d.alpha[i] = 1;
            d.line_alpha[i] = 1;
            d.tooltip[i] = makeTooltip(
                d.division[i],
                d.state[i],
                d.year[i],
                d.party[i],
                d.percent[i],
                d.candidate[i]
            );
        }
        source.change.emit();
    }

    // --- Fix: Clear selection and reset on empty click ---
    if (indices.length === 0) {
        resetAllPoints();
        selected_division_source.data = {x:[], y:[], year:[], color:[], tooltip:[]};
        selected_line_source.data = {xs:[], ys:[]};
        source.selected.indices = [];
        return;
    }

    let tapped_idx = (indices.length > 0) ? indices[0] : null;
    let tapped_division = (tapped_idx !== null) ? d.division[tapped_idx] : null;

    if (show_changing.active.length > 0) {
        show_changing.active = [];
        change_line_source.data = {xs: [], ys: []};
        resetAllPoints();
        selected_division_source.data = {x:[], y:[], year:[], color:[], tooltip:[]};
        selected_line_source.data = {xs:[], ys:[]};
        return;
    }

    const idx = indices[0];
    const division = d.division[idx];

    for (let i = 0; i < d.division.length; i++) {
        if (d.division[i] === division) {
            d.alpha[i] = 1;
            d.line_alpha[i] = 1;
            d.tooltip[i] = "";
        } else {
            d.alpha[i] = 0.1;
            d.line_alpha[i] = 0.1;
            d.tooltip[i] = "";
        }
    }
    source.change.emit();

    // --- Build division history from js_data_year ---
    let x = [], y = [], year = [], color = [], tooltip = [];
    for (let yidx = 0; yidx < election_years.length; yidx++) {
        let ystr = election_years[yidx];
        let ydata = js_data_year[ystr];
        let idx = ydata.division.indexOf(division);
        if (idx >= 0) {
            x.push(ydata.x[idx]);
            y.push(ydata.y[idx]);
            year.push(ystr);
            color.push(ydata.color[idx]);
            tooltip.push(makeTooltip(
                ydata.division[idx],
                ydata.state[idx],
                ystr,
                ydata.party[idx],
                ydata.percent[idx],
                ydata.candidate[idx]
            ));
        }
    }
    selected_division_source.data = {x, y, year, color, tooltip};

    let xs = [], ys = [];
    for (let i = 1; i < x.length; i++) {
        xs.push([x[i-1], x[i]]);
        ys.push([y[i-1], y[i]]);
    }
    selected_line_source.data = {xs, ys};
    """
))

# --- Show Changing Seats callback ---
# ...existing code...

show_changing.js_on_change(
    "active", 
    bkm.CustomJS(
        args=dict(
            source= source,
            js_data_year= js_data_year,
            show_changing= show_changing,
            change_line_source= change_line_source,
            election_years= election_years,
            renderer=r,
            selected_division_source=selected_division_source,
            selected_line_source=selected_line_source,
        ), 
        code="""
    let d = source.data;
    const current_year = d.year[0];
    const year_data = js_data_year[current_year];

    function makeTooltip(division, state, year, parties, percents, candidates) {
        let html = `<div class="vote-tooltip"><h3 style="margin-top:0">${division}, ${state}, ${year}</h3><table>`;
        html += '<tr><th>Party</th><th>Vote Share</th><th>Candidate</th></tr>';
        for (let i = 0; i < parties.length; i++) {
            html += `<tr><td>${parties[i]}</td><td>${percents[i].toFixed(2)}</td><td>${candidates[i]}</td></tr>`;
        }
        html += '</table></div>';
        return html;
    }

    // --- If "Show Changing Seats" is being activated, clear division selection ---
    if (show_changing.active.length > 0) {
        if (typeof source.selected !== "undefined") {
            source.selected.indices = [];
        }
        selected_division_source.data = {x:[], y:[], year:[], color:[], tooltip:[]};
        selected_line_source.data = {xs:[], ys:[]};
    }

    if (show_changing.active.length == 0) {
        d.alpha = d.alpha.map(() => 1);
        d.line_alpha = d.line_alpha.map(() => 1);
        // Rebuild tooltips on the fly
        for (let i = 0; i < d.division.length; i++) {
            d.tooltip[i] = makeTooltip(
                d.division[i],
                d.state[i],
                d.year[i],
                d.party[i],
                d.percent[i],
                d.candidate[i]
            );
        }
        source.change.emit();
        change_line_source.data = {xs: [], ys: []};
        return;
    }

    const idx = election_years.indexOf(current_year);
    if (idx < 1) {
        d.alpha = d.alpha.map(() => 1);
        d.line_alpha = d.line_alpha.map(() => 1);
        // Rebuild tooltips on the fly
        for (let i = 0; i < d.division.length; i++) {
            d.tooltip[i] = makeTooltip(
                d.division[i],
                d.state[i],
                d.year[i],
                d.party[i],
                d.percent[i],
                d.candidate[i]
            );
        }
        source.change.emit();
        change_line_source.data = {xs: [], ys: []};
        return;
    }

    const prev_year = election_years[idx - 1];
    const prev = js_data_year[prev_year];
    let prev_lookup = {};
    for (let i = 0; i < prev.division.length; i++) {
        prev_lookup[prev.division[i]] = {
            x: prev.x[i],
            y: prev.y[i],
            color: prev.color[i]
        };
    }
    let xs = [], ys = [];
    for (let i = 0; i < d.division.length; i++) {
        const div = d.division[i];
        if (prev_lookup[div]) {
            const color_now = d.color[i];
            const color_prev = prev_lookup[div].color;
            const is_coalition_swap = (
                (color_now === '#1B7200' && color_prev === '#0000FF') ||
                (color_now === '#0000FF' && color_prev === '#1B7200')
            );
            if (color_now !== color_prev && !is_coalition_swap) {
                d.alpha[i] = 1;
                d.line_alpha[i] = 1;
                // Build tooltip on the fly
                d.tooltip[i] = makeTooltip(
                    d.division[i],
                    d.state[i],
                    d.year[i],
                    d.party[i],
                    d.percent[i],
                    d.candidate[i]
                );
                xs.push([prev_lookup[div].x, d.x[i]]);
                ys.push([prev_lookup[div].y, d.y[i]]);
            } else {
                d.alpha[i] = 0.1;
                d.line_alpha[i] = 0.1;
                d.tooltip[i] = "";
            }
        } else {
            d.alpha[i] = 0.1;
            d.line_alpha[i] = 0.1;
            d.tooltip[i] = "";
        }
    }
    source.change.emit();
    change_line_source.data = {xs: xs, ys: ys};
    """
    )
)

select.js_on_change(
    "value",
    bkm.CustomJS(
        args=dict(
            source=source,
            js_data_year=js_data_year,
            select=select,
            show_changing=show_changing,
            change_line_source=change_line_source,
            selected_division_source=selected_division_source,
            selected_line_source=selected_line_source,
        ),
        code="""
        function makeTooltip(division, state, year, parties, percents, candidates) {
            let html = `<div class="vote-tooltip"><h3 style="margin-top:0">${division}, ${state}, ${year}</h3><table>`;
            html += '<tr><th>Party</th><th>Vote Share</th><th>Candidate</th></tr>';
            for (let i = 0; i < parties.length; i++) {
                html += `<tr><td>${parties[i]}</td><td>${percents[i].toFixed(2)}</td><td>${candidates[i]}</td></tr>`;
            }
            html += '</table></div>';
            return html;
        }

        // Reset UI state
        show_changing.active = [];
        source.selected.indices = [];
        change_line_source.data = {xs: [], ys: []};
        selected_division_source.data = {x:[], y:[], year:[], color:[], tooltip:[]};
        selected_line_source.data = {xs:[], ys:[]};

        // Get years and data
        const old_year = source.data.year[0];
        const new_year = select.value;
        const old_data = js_data_year[old_year];
        const new_data = js_data_year[new_year];

        // Build division lookup for both years
        let old_lookup = {};
        for (let i = 0; i < old_data.division.length; i++) {
            old_lookup[old_data.division[i]] = i;
        }
        let new_lookup = {};
        for (let i = 0; i < new_data.division.length; i++) {
            new_lookup[new_data.division[i]] = i;
        }

        // Build union of all divisions
        let all_divisions = Array.from(new Set(
            old_data.division.concat(new_data.division)
        ));

        // Prepare start and end states for animation
        let start_x = [], start_y = [], start_color = [], start_alpha = [];
        let start_party = [], start_percent = [], start_candidate = [], start_state = [], start_tooltip = [], start_year = [];
        let end_x = [], end_y = [], end_color = [], end_alpha = [];
        let end_party = [], end_percent = [], end_candidate = [], end_state = [], end_tooltip = [], end_year = [];

        for (let i = 0; i < all_divisions.length; i++) {
            let div = all_divisions[i];
            let old_idx = old_lookup[div];
            let new_idx = new_lookup[div];

            // Start state
            if (old_idx !== undefined) {
                start_x.push(old_data.x[old_idx]);
                start_y.push(old_data.y[old_idx]);
                start_color.push(old_data.color[old_idx]);
                start_alpha.push(1);
                start_party.push(old_data.party[old_idx]);
                start_percent.push(old_data.percent[old_idx]);
                start_candidate.push(old_data.candidate[old_idx]);
                start_state.push(old_data.state[old_idx]);
                start_tooltip.push(makeTooltip(
                    old_data.division[old_idx],
                    old_data.state[old_idx],
                    old_year,
                    old_data.party[old_idx],
                    old_data.percent[old_idx],
                    old_data.candidate[old_idx]
                ));
                start_year.push(old_year);
            } else {
                // Division is new, fade in from invisible at new position
                start_x.push(new_data.x[new_idx]);
                start_y.push(new_data.y[new_idx]);
                start_color.push(new_data.color[new_idx]);
                start_alpha.push(0);
                start_party.push(new_data.party[new_idx]);
                start_percent.push(new_data.percent[new_idx]);
                start_candidate.push(new_data.candidate[new_idx]);
                start_state.push(new_data.state[new_idx]);
                start_tooltip.push(makeTooltip(
                    new_data.division[new_idx],
                    new_data.state[new_idx],
                    new_year,
                    new_data.party[new_idx],
                    new_data.percent[new_idx],
                    new_data.candidate[new_idx]
                ));
                start_year.push(new_year);
            }

            // End state
            if (new_idx !== undefined) {
                end_x.push(new_data.x[new_idx]);
                end_y.push(new_data.y[new_idx]);
                end_color.push(new_data.color[new_idx]);
                end_alpha.push(1);
                end_party.push(new_data.party[new_idx]);
                end_percent.push(new_data.percent[new_idx]);
                end_candidate.push(new_data.candidate[new_idx]);
                end_state.push(new_data.state[new_idx]);
                end_tooltip.push(makeTooltip(
                    new_data.division[new_idx],
                    new_data.state[new_idx],
                    new_year,
                    new_data.party[new_idx],
                    new_data.percent[new_idx],
                    new_data.candidate[new_idx]
                ));
                end_year.push(new_year);
            } else {
                // Division is old, fade out at old position
                end_x.push(old_data.x[old_idx]);
                end_y.push(old_data.y[old_idx]);
                end_color.push(old_data.color[old_idx]);
                end_alpha.push(0);
                end_party.push(old_data.party[old_idx]);
                end_percent.push(old_data.percent[old_idx]);
                end_candidate.push(old_data.candidate[old_idx]);
                end_state.push(old_data.state[old_idx]);
                end_tooltip.push(makeTooltip(
                    old_data.division[old_idx],
                    old_data.state[old_idx],
                    old_year,
                    old_data.party[old_idx],
                    old_data.percent[old_idx],
                    old_data.candidate[old_idx]
                ));
                end_year.push(old_year);
            }
        }

        // Animation
        let frames = 20;
        let frame = 0;
        function animate() {
            let t = frame / frames;
            let x = [], y = [], color = [], alpha = [];
            let party = [], percent = [], candidate = [], state = [], tooltip = [], division = [], year = [];
            for (let i = 0; i < all_divisions.length; i++) {
                x.push(start_x[i] + (end_x[i] - start_x[i]) * t);
                y.push(start_y[i] + (end_y[i] - start_y[i]) * t);
                color.push(end_color[i]);
                alpha.push(start_alpha[i] + (end_alpha[i] - start_alpha[i]) * t);
                party.push(end_party[i]);
                percent.push(end_percent[i]);
                candidate.push(end_candidate[i]);
                state.push(end_state[i]);
                tooltip.push(end_tooltip[i]);
                division.push(all_divisions[i]);
                year.push(end_year[i]);
            }
            source.data = {
                x: x,
                y: y,
                color: color,
                tooltip: tooltip,
                division: division,
                year: year,
                alpha: alpha,
                line_alpha: alpha,
                party: party,
                percent: percent,
                candidate: candidate,
                state: state
            };
            source.change.emit();
            frame++;
            if (frame <= frames) {
                requestAnimationFrame(animate);
            } else {
                // After animation, set source.data to only the new year's divisions
                source.data = {
                    division: new_data.division,
                    x: new_data.x,
                    y: new_data.y,
                    color: new_data.color,
                    tooltip: new_data.division.map((div, i) =>
                        makeTooltip(
                            new_data.division[i],
                            new_data.state[i],
                            new_year,
                            new_data.party[i],
                            new_data.percent[i],
                            new_data.candidate[i]
                        )
                    ),
                    state: new_data.state,
                    party: new_data.party,
                    percent: new_data.percent,
                    candidate: new_data.candidate,
                    year: Array(new_data.division.length).fill(new_year),
                    alpha: Array(new_data.division.length).fill(1),
                    line_alpha: Array(new_data.division.length).fill(1)
                };
                source.change.emit();
            }
        }
        animate();
        """
    )
)

# JS callback for prev/next buttons
button_prev.js_on_click(
    bkm.CustomJS(
        args=dict(
            select=select,
            years=election_years
        ),
        code="""
        const idx = years.indexOf(select.value);
        if (idx > 0) {
            select.value = years[idx - 1];
        }
        """
    )
)
button_next.js_on_click(
    bkm.CustomJS(
        args=dict(
            select=select,
            years=election_years
        ),
        code="""
        const idx = years.indexOf(select.value);
        if (idx < years.length - 1) {
            select.value = years[idx + 1];
        }
        """
    )
)

layout = bkhrow(p, bkhcolumn(bkhrow(button_prev, select, button_next), show_changing))

show(layout)

In [ ]:
bkp.save(layout, 'index.html', title='AusFedPol First Preferences')